### <$Q_\pi$를 추정하기 위한  n 단계 트리 보강 > 

모든 $s \in S, a \in Q$ 에 대해 Q(s,a)를 임의의 값으로 초기화 
정책 $\pi$를 Q에 대해 탐욕적인 정책이 되도록 초기화, 또는 주어진 고정된 정책으로 초기화 
알고리즘 파라미터 : 시간 간격 $\alpha \in (0,1] $, 양의 정수 $\alpha$  
($S_t$와 $A_t$, $R_t$에 대한) 모든 저장과 접근은 n+1로 인텍스를 설정하여 행해질 수 있음. 

각 에피소드에 대한 루프 : 
- $S_0$를 초기화하고 저장(단, $S_0 \neq$ 종단) 
- 행동 $A_0 ~ b(.|S_a)$를 선택하고 저장  
- $ T <- $ 무한대
- t= 0,1,2, ... 에 대한 루프 : 

> t < T 이면 : 
 > -  행동 $A_t$를 취함
 > - 다음 보상을 $R_{t+1}$, 다음 상태를 $S_{t+1}$로 측정하고 저장함 
 > - $S_{t+1}$ 이 종단이면, T <- t+1 
 > - 종단이 아니면, 행동 $A_{t+1} ~ b(.|S_{t+1})$을 선택하고 저장 
 
> r <- t-n+1(r은 상태의 추정값이 갱신되는 시각) 

> r>=0 이면 : 
> - t +1 >= T 이면 :  G <- $R_T$ 

> - t+1 < T 이면 : 

> > G <-  $R_{t+1} + \gamma \sum_a \pi(a|S_{t+1}) Q(S_{t+1}, a)$ 

> > k = min(t,T-1) 부터 r+1 까지의 루프 : 
 G <- $R_k + \gamma \sum_{a \neq A_k} \pi(a|S_k)Q(S_k,a) + \gamma \pi(A_k|S_k)G$ 
 
> > $Q(S_r, A_r)$ <- $Q(S_r, A_r) + \alpha [G-Q(S_r, A_r)]$

> > $\pi$를 학습한다면 $\pi(.|S_r)$ 가 Q에 대해 탐욕적이 되도록 해야 함. 

- r = T-1 이면 종료


**<구현해야 하는 것>**

**<필요한 것>** 
- $\alpha$ : class 제작시 입력 값으로 부여 
- n : 몇단계 TD를 할 것인가? 

**<함수 / 데이터 형식>** 
- class evaluate_Q_n_TD : 
> def __init__(self, S, A, n, b, alpha, reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 

**<외부함수>** 
- R(s',a,s) : 보상함수

**<고민점>** 


In [4]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [6]:
from collections import defaultdict
import random
import numpy as np

In [7]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 

def choose_random_max(lst) : #최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a의 인덱스를 추출 
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 



In [14]:
# n 단계 Q값 추정 코드 참고 

class tree_evaluate_Q_n_TD :  
    def __init__(self, S, A, n, reward_func, alpha=0.1,  epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.n = n
        self.reward_func = reward_func
        self.alpha = alpha
        self.epsilon = epsilon 
        self.gamma = gamma 
        self.num_episode = num_episode
        self.T = len_episode

        self.Q = self.initiate_Q() 
#        self.V = self.initiate_V() 
    
    def initiate_Q(self) : # Q(s,a) 값을 초기화 
        Q_dict = defaultdict(float)
        for s in self.S : 
            for a in self.A : 
                Q_dict[(s,a)] = 0         
        
        return Q_dict

    def pi(self, s) : #Q 값을 기반으로 하여 "탐욕적" 선택하도록 확률 값을 반환하기.  
        lst = [] 
        for a in self.A : lst.append(self.Q[(s,a)])
        max_a = choose_random_max(lst)
        prob_s = [0]*len(self.A) 
        prob_s[max_a] = 1 
        return prob_s
    
    def choice_action(self, s, policy) : #수정 - pi 함수를 재정의한 것과 맞춰 수정 
        a= random.choices(self.A, weights = policy(s))
        a = a[0]
        return a
    
    def next_s(self, s,a) : # 상태 s에서 a 행동을 했을 때 다음 상태 s'. 정책, S,A 에 따라 달라짐. 
        return min(max(s+a, 0), max(self.S)) 


    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []} 
        episode["R"].append(0) # R_0 값 부여
        episode["S"].append(start_s) #S_0 값 부여.
        for _ in range(T) : 
            episode["S"].append(s)
            a = self.choice_action(s, self.pi) 
            next_s = self.next_s(s, a)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        episode["R"].append(0) # R_{t+1} =0 값 추가.  / S_{t+1} 도 추가해야할 것 같은데
        episode["S"].append(self.next_s(s,a))
        return episode 
 
    def update_returns(self) : # 
        
        for _ in range(self.num_episode) : 
            start_s = random.choice(self.S) 
            T = self.T # 어차피 T는 self.T로 갱신되기 때문에 앞에서 설정함. 
            episode = self.make_episode(start_s, self.T) 
            S,R,A = episode['S'],episode['R'], episode['A'] 
            
            for t in range(self.T + self.n +1) : # t는 아무리 커도 self.T + n 보다 커질 수 없기 때문 
                # make_episode 에서 이미 a,r를 계산해 두었기 때문에 t<T 조건문 수도코드 구현 생략. 
                
                # r 또한 현재 t와 인덱스로 사용되는 n이 중복되기 때문에, t값이 T 값일 때 순환문을 종료하는 것으로 
                
                r = t - self.n +1 
                if r >= 0 : 
                    if t+ 1 >= T : G = R[self.T]
                    else : 
                        G = R[t+1] + self.gamma * sum([self.pi(S[t])[self.A.index(i)] * self.Q[(S[t], i)] for i in self.A ])
                        for k in range(min(t,T-1), r+2) : 
                            G = R[k] + sum([self.pi(S[t])[self.A.index(i)] * self.Q[(S[t], i)] for i in self.A if i != A[t]]) + self.gamma * self.pi(S[t+1])[self.A.index(A[r])]*G 
                        self.Q[(S[r], A[r])] = self.Q[(S[r], A[r])] + self.alpha * (G - self.Q[(S[r], A[r])])
                        
        
        
         

In [15]:
test = tree_evaluate_Q_n_TD(S,A, 10, reward_func)
test.update_returns() 
#print(test.Q)

for _ in range(1000) : test.update_returns()  
print(test.Q)

# 확실히 비활성 정책으로 하는게 여러가지 경우를 포함하면서 학습을 잘하는 듯 

defaultdict(<class 'float'>, {(0, -5): -0.1355, (0, -4): -0.0799073520523153, (0, -3): -0.020000000000000004, (0, -2): -0.26599660799930147, (0, -1): -0.52012472023, (0, 0): -1.8758002375633845e-16, (0, 1): -0.084107080413667, (0, 2): -0.12598011360253, (0, 3): -0.05877028414548097, (0, 4): -0.02378250972962712, (1, -5): -0.12309033333333333, (1, -4): 1.4157344718342672e-05, (1, -3): -0.06283603001444704, (1, -2): -0.0003399037552077712, (1, -1): -0.12594560517198153, (1, 0): -4.356980097050541e-07, (1, 1): -0.009568646846653803, (1, 2): -6.581004679378707e-05, (1, 3): -0.08688596524961835, (1, 4): -0.025, (2, -5): 0, (2, -4): -0.004730945931989244, (2, -3): -0.1742270924142462, (2, -2): 0.00011163649930227977, (2, -1): 0, (2, 0): -3.4286529704945655e-07, (2, 1): -0.004552427013002326, (2, 2): -0.022000512915369794, (2, 3): 0.00010141411846570262, (2, 4): -0.06752951034801911, (3, -5): 0, (3, -4): 0.0015672276465060826, (3, -3): 0, (3, -2): -0.0014018423260055432, (3, -1): 0.0016406257